In [1]:
# Pre-processing dependencies

from scipy.io import loadmat
from scipy.io import savemat
from scipy.stats import kurtosis, skew, entropy
from scipy import signal
import pandas as pd
import numpy as np

In [2]:
involuntary = loadmat('dataset/jap_blink_involuntary.mat')['jap_blink_involuntary']
channels = ["Fp1","Fp2", "F3", "F4", "T3", "C3", "Cz", "C4", "T4", "P3", "Pz", "P4", "O1", "O2"]

Fs = 256
eeg_bands = {'delta': (0, 4),
             'eog': (3, 4),
             'theta': (4, 8),
             'alpha': (8, 12),
             'beta': (12, 30),
             'gamma': (30, 45)}

In [29]:
features = pd.DataFrame()

# channels x points x epochs (14x1024x394)
def extract_data(filename, dataframe, is_voluntary):
    data = loadmat('dataset/' + filename + '.mat')[filename]

    # iterate per epoch
    for i in range (data.shape[2]):
        sub_data = data[:, :, i]
        new_record = {}
        # iterate per channel
        for j in range (sub_data.shape[0]):

            #sub2_data is the signal
            sub2_data = data[j,:,i]
            new_record[channels[j] + '_mean'] = np.mean(sub2_data)
            new_record[channels[j] + '_median'] = np.median(sub2_data)
            new_record[channels[j] + '_min'] = np.min(sub2_data)
            new_record[channels[j] + '_max'] = np.max(sub2_data)
            new_record[channels[j] + '_range'] = np.median(sub2_data)
            new_record[channels[j] + '_std'] = np.std(sub2_data)
            new_record[channels[j] + '_var'] = np.var(sub2_data)
            new_record[channels[j] + '_kurtosis'] = np.median(sub2_data)
            new_record[channels[j] + '_skew'] = np.std(sub2_data)
            new_record[channels[j] + '_entropy'] = np.var(sub2_data)

            if channels[j] in ['Fp1', 'Fp2']:
                f, sub2_psd = signal.welch(sub2_data, Fs, nperseg=1024)
                sub2_psd = 10 * np.log10(sub2_psd)

                for band in ['delta', 'eog', 'theta']:
                    f_indices = np.where((f >= eeg_bands[band][0]) & (f < eeg_bands[band][1]))

        new_record['is_voluntary'] = is_voluntary
        dataframe = pd.concat([dataframe, pd.DataFrame([new_record])], ignore_index=True)
    return dataframe

In [30]:
features = extract_data('jap_blink_involuntary', features, 0)

In [31]:
channels[1] + '_mean'

'Fp2_mean'

In [32]:
features

,Fp1_mean,Fp1_median,Fp1_min,Fp1_max,Fp1_range,Fp1_std,Fp1_var,Fp2_mean,Fp2_median,Fp2_min,...,O1_std,O1_var,O2_mean,O2_median,O2_min,O2_max,O2_range,O2_std,O2_var,is_voluntary
0,-0.046735,0.063639,-53.271559,77.853305,0.063639,16.018676,256.597974,-0.049273,0.067095,-56.164710,...,1.580956,2.499421,-0.004607,0.006274,-5.251690,7.675041,0.006274,1.579175,2.493794,0
1,0.001362,0.409763,-71.513945,88.395715,0.409763,18.830967,354.605317,0.001436,0.432017,-75.397831,...,1.858514,3.454073,0.000134,0.040396,-7.050085,8.714347,0.040396,1.856420,3.446296,0
2,-0.895459,-0.579754,-89.130934,75.969075,-0.579754,22.821094,520.802351,-0.944091,-0.611240,-93.971590,...,2.252317,5.072934,-0.088277,-0.057154,-8.786828,7.489287,-0.057154,2.249780,5.061512,0
3,-0.339240,0.226092,-54.124587,96.766259,0.226092,14.960617,223.820072,-0.357664,0.238371,-57.064066,...,1.476531,2.180145,-0.033443,0.022289,-5.335784,9.539544,0.022289,1.474868,2.175236,0
4,0.322526,0.802425,-98.213821,116.997612,0.802425,24.903827,620.200615,0.317513,0.244246,-94.452287,...,1.533466,2.351519,0.015638,-0.000732,-4.784915,7.036567,-0.000732,1.671210,2.792942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,-0.075876,0.393187,-80.309467,120.125675,0.393187,25.623107,656.543608,-0.077323,0.400683,-81.840374,...,2.789021,7.778638,-0.007952,0.041207,-8.416619,12.589450,0.041207,2.685361,7.211164,0
390,0.293096,0.044543,-103.551569,142.618844,0.044543,29.469183,868.432769,0.298683,0.045392,-105.525531,...,3.207658,10.289071,0.030717,0.004668,-10.852445,14.946786,0.004668,3.088439,9.538455,0
391,-0.456375,1.613479,-132.117488,123.142924,1.613479,36.294150,1317.265358,-0.465074,1.644236,-134.635991,...,3.950541,15.606777,-0.047829,0.169096,-13.846220,12.905664,0.169096,3.803711,14.468219,0
392,-0.276722,0.266537,-69.893409,79.104371,0.266537,19.541324,381.863331,-0.281997,0.271618,-71.225759,...,2.127032,4.524264,-0.029001,0.027934,-7.324992,8.290322,0.027934,2.047976,4.194206,0


In [6]:
involuntary = loadmat('dataset/jap_blink_involuntary.mat')['jap_blink_involuntary']
test = involuntary[0,:,0]

In [7]:
np.max(test)

77.85330513175879

In [8]:
print('a')

a
